In [5]:
# sympy 버전을 transformers와 호환되는 안정된 버전으로 다운그레이드
!pip install sympy==1.12 --quiet
# 런타임 재시작 필요 없이 바로 사용하도록 transformers 재설치
!pip install transformers --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 54.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-runtime-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-runtime-cu12 12.5.82 which is incompati

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from google.colab import drive

# ✅ Google Drive 마운트
drive.mount('/content/drive')

# ✅ 모델 경로
model_path = "/content/drive/MyDrive/models/codebert_finetuned_v2"

# ✅ 모델과 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

print("✅ Google Drive에서 모델 불러오기 완료!")

Mounted at /content/drive
✅ Google Drive에서 모델 불러오기 완료!


In [2]:
# 최신 openai (v1 이상) 방식
from openai import OpenAI

# ⚠️ 파일 업로드할 때는 숨기기
client = OpenAI(api_key="GPT-API Key")

def generate_report(code_snippet: str, label: str) -> str:
    prompt = f"""
당신은 보안 분석 전문가입니다.
아래의 소스 코드는 '{label}' 취약점이 감지된 코드입니다.

[취약 코드]
{code_snippet}

이 코드에 대해 다음 내용을 자세히 설명해주세요:
1. 취약점 설명 (왜 위험한가?)
2. 공격자가 악용하는 방법
3. 보완 방법 (보완된 코드 예시 포함)
4. 요약된 보안 권고사항

전문적인 보안 분석 보고서 형태로 출력해주세요.
"""

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant for code security analysis."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,
        max_tokens=2048
    )

    return response.choices[0].message.content

In [37]:
import torch

# ✅ 라벨 및 점수 매핑 딕셔너리
score_map = {
    0: {"label": "SQL_Injection", "score": 30, "msg": "⚠️ SQL Injection 취약점 감지"},
    1: {"label": "Hardcoded_Password", "score": 30, "msg": "⚠️ Hardcoded Password 감지"},
    2: {"label": "XSS", "score": 30, "msg": "⚠️ XSS 취약점 감지"},
    3: {"label": "Other", "score": 70, "msg": "⚠️ 잠재적 취약점이 있는 코드 (Other)"},
    4: {"label": "Safe_Code", "score": 100, "msg": "✅ 완전한 방어 코드 (Safe Code)"}
}

def analyze_code(code_snippet):
    inputs = tokenizer(
        code_snippet,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128
    )

    # ✅ 모델과 입력 텐서를 동일한 장치로 이동 (GPU 또는 CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # ✅ 예측 수행
    with torch.no_grad():
        outputs = model(**inputs)
        predicted = torch.argmax(outputs.logits, dim=1).item()

    # ✅ 결과 리턴
    result = score_map[predicted]
    return {
        "prediction": result["msg"],
        "label": result["label"],
        "security_score": result["score"]
    }

In [38]:
# 5가지 레이블 테스트용 코드 샘플
test_cases = [
    # 1: SQL Injection
    """
    user_input = request.args.get('username')
    query = "SELECT * FROM users WHERE name = '" + user_input + "'"
    """,

    # 2: Hardcoded Password
    """
    password = "admin123"
    login(user, password)
    """,

    # 3: XSS
    """
    @app.route('/search')
    def search():
        keyword = request.args.get('q')
        return "<p>" + keyword + "</p>"
    """,

    # 4: Safe Code
    """
    user_input = request.args.get('username')
    query = "SELECT * FROM users WHERE name = %s"
    cursor.execute(query, (user_input,))
    """,

    # 5: Other
    """
    with open("log.txt", "w") as f:
        f.write("User logged in")
    """
]

# 결과 출력
for idx, code in enumerate(test_cases, start=1):
    result = analyze_code(code)
    print(f"[Test Case {idx}] 🔍 결과: {result['prediction']} | 🔐 점수: {result['security_score']}")

[Test Case 1] 🔍 결과: ⚠️ SQL Injection 취약점 감지 | 🔐 점수: 30
[Test Case 2] 🔍 결과: ⚠️ Hardcoded Password 감지 | 🔐 점수: 30
[Test Case 3] 🔍 결과: ⚠️ XSS 취약점 감지 | 🔐 점수: 30
[Test Case 4] 🔍 결과: ✅ 완전한 방어 코드 (Safe Code) | 🔐 점수: 100
[Test Case 5] 🔍 결과: ✅ 완전한 방어 코드 (Safe Code) | 🔐 점수: 100


In [8]:
# 데이터 추가 학습용 파일 업로드
from google.colab import files
import pandas as pd

uploaded = files.upload()

# 업로드한 CSV 파일 로드
df = pd.read_csv("SQL_Injection_Examples.csv")

# 확인
print("✅ CSV 로드 완료!")
print(df.head())

Saving SQL_Injection_Examples.csv to SQL_Injection_Examples (1).csv
✅ CSV 로드 완료!
                                        code_snippet          label
0  user_input = request.args.get('username')\nque...  SQL_Injection
1  query = "SELECT * FROM products WHERE id = " +...  SQL_Injection
2  sql = "DELETE FROM orders WHERE order_id = " +...  SQL_Injection
3  username = input("Username: ")\nsql = "SELECT ...  SQL_Injection
4  query = "INSERT INTO logs (message) VALUES ('"...  SQL_Injection


In [9]:
!pip install -U datasets
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [19]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

# ✅ CSV 파일 불러오기
df = pd.read_csv("SQL_Injection_Examples.csv")

# ✅ 라벨 매핑
label_map = {
    "SQL_Injection": 0,
    "Hardcoded_Password": 1,
    "XSS": 2,
    "Safe_Code": 3,
    "Other": 4
}
df["label"] = df["label"].map(label_map)

# ✅ NaN 제거
df.dropna(inplace=True)

# ✅ CodeBERT 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")

# ✅ 토크나이즈 함수 정의
def tokenize(example):
    tokens = tokenizer(example["code_snippet"], padding="max_length", truncation=True, max_length=128)
    tokens["labels"] = example["label"]
    return tokens

# ✅ Hugging Face Dataset으로 변환 & 전처리
dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(tokenize)

# ✅ 기존 파인튜닝된 모델 로드
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/models/codebert_finetuned_v2", num_labels=5)

# ✅ 학습 설정
training_args = TrainingArguments(
    output_dir="./codebert_finetuned_v3",  # ✅ 학습 결과 저장 위치
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=10,
    report_to="none"
)

# ✅ Trainer 정의 및 학습 실행
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset
)

trainer.train()

# ✅ 모델 저장
model.save_pretrained("./codebert_finetuned_v3")
tokenizer.save_pretrained("./codebert_finetuned_v3")
print("✅ SQL Injection 예제 추가 학습 및 저장 완료!")

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.130895
2,No log,0.088731
3,No log,0.075988


✅ SQL Injection 예제 추가 학습 및 저장 완료!


In [15]:
# 데이터 추가 학습용 파일 업로드
from google.colab import files
import pandas as pd

uploaded = files.upload()

# 업로드한 CSV 파일 로드
df = pd.read_csv("ml_examples.csv")

# 확인
print("✅ CSV 로드 완료!")
print(df.head())

Saving ml_examples.csv to ml_examples.csv
✅ CSV 로드 완료!
                                        code_snippet label
0  @app.route('/search')\ndef search():\n    quer...   XSS
1  from flask import Flask, request\napp = Flask(...   XSS
2  @app.route('/comment')\ndef comment():\n    us...   XSS
3  @app.route('/search')\ndef search():\n    quer...   XSS
4  from flask import Flask, request\napp = Flask(...   XSS


In [24]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

# ✅ CSV 파일 불러오기
df = pd.read_csv("ml_examples.csv")

# ✅ 라벨 매핑
label_map = {
    "SQL_Injection": 0,
    "Hardcoded_Password": 1,
    "XSS": 2,
    "Safe_Code": 3,
    "Other": 4
}
df["label"] = df["label"].map(label_map)

# ✅ NaN 제거
df.dropna(inplace=True)

# ✅ CodeBERT 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")

# ✅ 토크나이즈 함수 정의
def tokenize(example):
    tokens = tokenizer(example["code_snippet"], padding="max_length", truncation=True, max_length=128)
    tokens["labels"] = example["label"]
    return tokens

# ✅ Hugging Face Dataset으로 변환 & 전처리
dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(tokenize)

# ✅ ✅ ✅ Colab 내부에 저장된 모델 경로로 변경
model_path = "./codebert_finetuned_v3"

# ✅ 기존 파인튜닝된 모델 로드
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=5)

# ✅ 학습 설정
training_args = TrainingArguments(
    output_dir="./codebert_finetuned_v4",  # ✅ 학습 결과 저장 위치
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=10,
    report_to="none"
)

# ✅ Trainer 정의 및 학습 실행
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset
)

trainer.train()

# ✅ 모델 저장
model.save_pretrained("./codebert_finetuned_v4")
tokenizer.save_pretrained("./codebert_finetuned_v4")

print("✅ 취약점 예제 추가 학습 및 저장 완료!")

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.733576
2,1.058000,0.525765
3,0.663800,0.437857


✅ 취약점 예제 추가 학습 및 저장 완료!


In [39]:
# ✅ 현재 학습한 모델을 저장할 경로 지정
save_path = "/content/drive/MyDrive/models/codebert_finetuned_v4"
# ✅ 모델과 토크나이저 저장
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print("✅ 모델이 Google Drive에 성공적으로 저장되었습니다!")

✅ 모델이 Google Drive에 성공적으로 저장되었습니다!


In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from google.colab import drive

# ✅ Google Drive 마운트
drive.mount('/content/drive')

# ✅ 모델 경로
model_path = "/content/drive/MyDrive/models/codebert_finetuned_v4"

# ✅ 모델과 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

print("✅ Google Drive에서 모델 불러오기 완료!")

Mounted at /content/drive
✅ Google Drive에서 모델 불러오기 완료!


In [2]:
# 최신 openai (v1 이상) 방식
from openai import OpenAI

# ⚠️ 파일 업로드할 때는 숨기기
client = OpenAI(api_key="GPT-API Key")

def generate_report(code_snippet: str, label: str) -> str:
    prompt = f"""
당신은 보안 분석 전문가입니다.
아래의 소스 코드는 '{label}' 취약점이 감지된 코드입니다.

[취약 코드]
{code_snippet}

이 코드에 대해 다음 내용을 자세히 설명해주세요:
1. 취약점 설명 (왜 위험한가?)
2. 공격자가 악용하는 방법
3. 보완 방법 (보완된 코드 예시 포함)
4. 요약된 보안 권고사항

전문적인 보안 분석 보고서 형태로 출력해주세요.
"""

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant for code security analysis."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,
        max_tokens=2048
    )

    return response.choices[0].message.content

In [4]:
import torch
import torch.nn.functional as F

# ✅ 라벨 및 점수 매핑 딕셔너리
score_map = {
    0: {"label": "SQL_Injection", "score": 30, "msg": "⚠️ SQL Injection 취약점 감지"},
    1: {"label": "Hardcoded_Password", "score": 30, "msg": "⚠️ Hardcoded Password 감지"},
    2: {"label": "XSS", "score": 30, "msg": "⚠️ XSS 취약점 감지"},
    3: {"label": "Safe_Code", "score": 100, "msg": "✅ 완전한 방어 코드 (Safe Code)"},
    4: {"label": "Other", "score": 70, "msg": "⚠️ 잠재적 취약점이 있는 코드 (Other)"}
}

def analyze_code(code_snippet):
    inputs = tokenizer(code_snippet, return_tensors="pt", truncation=True, padding="max_length", max_length=128)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = F.softmax(logits, dim=1)[0]  # 확률로 변환
        predicted = torch.argmax(probs).item()

        # 후처리 로직: Safe_Code vs Other 세분화
        if predicted == 3:  # Safe_Code로 예측됐을 때만 검사
            safe_prob = probs[3].item()
            other_prob = probs[4].item()
            if other_prob > 0.3 and safe_prob < 0.7:
                predicted = 4  # 확신이 부족하면 Other로 판단

    return {
        "prediction": score_map[predicted]["msg"],
        "label": score_map[predicted]["label"],
        "security_score": score_map[predicted]["score"]
    }

In [6]:
test_cases = [
    # 1: SQL Injection
    """
    user_input = request.args.get('username')
    query = "SELECT * FROM users WHERE name = '" + user_input + "'"
    """,

    # 2: Hardcoded Password
    """
    password = "admin123"
    login(user, password)
    """,

    # 3: XSS
    """
    @app.route('/search')
    def search():
        keyword = request.args.get('q')
        return "<p>" + keyword + "</p>"
    """,

    # 4: Safe Code
    """
    user_input = request.args.get('username')
    query = "SELECT * FROM users WHERE name = %s"
    cursor.execute(query, (user_input,))
    """,

    # 5: 로그 기록 (Other)
    """
    with open("log.txt", "w") as f:
        f.write("User logged in")
    """
]

# 결과 출력
for idx, code in enumerate(test_cases, start=1):
    result = analyze_code(code)
    print(f"[Test Case {idx}] 🔍 결과: {result['prediction']} | 🔐 점수: {result['security_score']}")

[Test Case 1] 🔍 결과: ⚠️ SQL Injection 취약점 감지 | 🔐 점수: 30
[Test Case 2] 🔍 결과: ⚠️ Hardcoded Password 감지 | 🔐 점수: 30
[Test Case 3] 🔍 결과: ⚠️ XSS 취약점 감지 | 🔐 점수: 30
[Test Case 4] 🔍 결과: ✅ 완전한 방어 코드 (Safe Code) | 🔐 점수: 100
[Test Case 5] 🔍 결과: ⚠️ 잠재적 취약점이 있는 코드 (Other) | 🔐 점수: 70


In [12]:
# 최신 openai (v1 이상) 방식
from openai import OpenAI

# ⚠️ 파일 업로드할 때는 숨기기
client = OpenAI(api_key="GPT-API Key")

# ✅ GPT 기반 리포트 생성 함수
def generate_report(code_snippet: str, label: str) -> str:
    prompt = f"""
당신은 보안 분석 전문가입니다.
아래의 소스 코드에서 감지된 취약점은 **'{label}'** 입니다.

[취약 코드]
{code_snippet.rstrip()}


📌 **1. 취약점 설명**
해당 취약점이 발생한 이유와 코드 구조상 문제점을 설명해주세요.

💣 **2. 공격 시나리오**
공격자가 해당 코드를 어떻게 악용할 수 있는지 설명해주세요.

🛠 **3. 보완 방법 및 개선된 코드 예시**
보완 방법을 설명하고, 보완된 코드 예시를 포함해주세요.

✅ **4. 요약된 보안 권고사항**
요약된 권고사항을 **리스트 형식**으로 정리해주세요.

---

아래 형식으로 리포트를 작성해주세요:

📄 **보안 분석 리포트**

---
"""

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant for code security analysis."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,
        max_tokens=1200  # 최대 출력 길이 여유 있게 확보
    )

    return response.choices[0].message.content

In [13]:
# 예제 코드 & 레이블
code = """
user_input = request.args.get('username')
query = "SELECT * FROM users WHERE name = '" + user_input + "'"
"""

label = "SQL_Injection"

# 보고서 생성
report = generate_report(code, label)
print(report)

📄 **보안 분석 리포트**

---

📌 **1. 취약점 설명**  
해당 코드에서 발생한 SQL Injection 취약점은 사용자로부터 입력받은 데이터를 쿼리에 그대로 삽입하는 방식으로 구현되어 있기 때문에 발생합니다. 사용자 입력이 쿼리에 직접 연결되므로 악의적인 SQL 코드가 주입될 수 있습니다.

💣 **2. 공격 시나리오**  
공격자는 사용자 입력란에 `' OR '1'='1' --`와 같은 SQL Injection 페이로드를 입력하여 WHERE 절을 항상 참으로 만들어 모든 사용자 데이터를 노출시킬 수 있습니다.

🛠 **3. 보완 방법 및 개선된 코드 예시**  
SQL Injection 공격을 방지하기 위해 사용자 입력을 안전하게 처리해야 합니다. 파라미터화된 쿼리나 ORM(Object-Relational Mapping)을 사용하여 SQL Injection 취약점을 방어할 수 있습니다. 아래는 개선된 코드 예시입니다.

```python
user_input = request.args.get('username')
query = "SELECT * FROM users WHERE name = %s"
cursor.execute(query, (user_input,))
```

위 코드에서는 사용자 입력을 파라미터로 전달하여 SQL 쿼리를 실행하므로 SQL Injection 공격을 방어할 수 있습니다.

✅ **4. 요약된 보안 권고사항**  
- 사용자 입력을 쿼리에 바로 삽입하지 말고 파라미터화된 쿼리를 사용하세요.
- ORM을 활용하여 데이터베이스 상호작용을 안전하게 처리하세요.
- 입력값을 검증하고 필요한 경우 이스케이프 함수를 사용하여 안전하게 처리하세요.
- 보안 취약점에 대한 교육 및 정기적인 코드 리뷰를 실시하여 취약점을 최소화하세요.
